# Data Analysis
## 저장된 데이터를 이용하여 백테스트 분석자료를 구성한다.
### 동시에 라이브 트레이딩의 분석자료 테스트용도로 사용한다

In [6]:
# 원인 없이 오류발생시 vscode를 재실행해야 한다.

In [1]:
import pickle
import os, sys
home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "github", "Thunder", "Binance"))

path_closing = os.path.join(home_path, "github", "TestData", "closing.pkl")
path_indices = os.path.join(home_path, "github", "TestData", "indices.pkl")

print(f"  🚀 storage 로딩 시작")
with open(path_closing, "rb")as file:
    storage_closing = pickle.load(file)
print(f"    📤 storage_closing 로딩완료")
with open(path_indices, "rb")as file:
    storage_indices = pickle.load(file)
print(f"    📤 storage_indices 로딩완료")
intervals = [i.split("_")[1] for i in storage_closing.__slots__]
print(f"    🔎 intervals 추출완료: {intervals}")
print(f"  ✅ 데이터 준비 완료")

  🚀 storage 로딩 시작
    📤 storage_closing 로딩완료
    📤 storage_indices 로딩완료
    🔎 intervals 추출완료: ['1m', '3m', '5m', '30m', '1h', '1d']
  ✅ 데이터 준비 완료


In [2]:
target_indieces = 100

indices_1m = storage_indices.get_data("1m", target_indieces)
indices_3m = storage_indices.get_data("3m", target_indieces)
indices_5m = storage_indices.get_data("5m", target_indieces)
indices_30m = storage_indices.get_data("30m", target_indieces)
indices_1h = storage_indices.get_data("1h", target_indieces)
indices_1d = storage_indices.get_data("1d", target_indieces)

data_1m = storage_closing.get_data('1m', indices_1m)
data_3m = storage_closing.get_data('3m', indices_3m)
data_5m = storage_closing.get_data('5m', indices_5m)
data_30m = storage_closing.get_data('30m', indices_5m)
data_1h = storage_closing.get_data('1h', indices_5m)
data_1d = storage_closing.get_data('1d', indices_5m)

print(f"  ✅ dummy data 준비 완료.")

  ✅ dummy data 준비 완료.


In [3]:
import Workspace.Analysis.Analysis as al
from Workspace.Analysis.Indicator import *

def cals_sma(values:np.ndarray, *args):
    result = {}
    for i in args:
        result[i] = MA.sma(values, i)
    return result

periods = (7, 25, 99)
sma_1m = cals_sma(data_1m[:, 4], *periods)
sma_30m = cals_sma(data_30m[:, 4], *periods)
sma_5m = cals_sma(data_5m[:, 4], *periods)
sma_1h = cals_sma(data_1h[:, 4], *periods)
closed_price = data_1m[:,4][-1]
up_m1 = closed_price < sma_1m[7][-1] < sma_1m[25][-1] < sma_1m[99][-1]
up_m30 = closed_price < sma_30m[7][-1] < sma_30m[25][-1] < sma_30m[99][-1]
up_m5 = closed_price < sma_5m[7][-1] < sma_5m[25][-1] < sma_5m[99][-1]
up_1h = closed_price < sma_1h[7][-1] < sma_1h[25][-1] < sma_1h[99][-1]

ema_m1_99 = np.all(np.diff(sma_1m[99]) < 0)
ema_m30_99 = np.all(np.diff(sma_30m[99]) < 0)
ema_m5_99 = np.all(np.diff(sma_5m[99]) < 0)
ema_1h_99 = np.all(np.diff(sma_1h[99]) < 0)

result:bool = all([up_m1, up_m30, up_m5, up_1h, ema_m1_99, ema_m30_99, ema_m5_99, ema_1h_99])

In [18]:
from typing import Optional, List
class Order_L:
    def __init__(self, target_roi:float):
        self.target_roi = target_roi
        self.status:bool = False
        self.entry_price:Optional[float] = None
        self.high_price:Optional[float] = None
        self.low_price:Optional[float] = None
        self.trade_history:List = []

    def update_low_price(self, value:float):
        if self.status:
            self.low_price = min(self.low_price, value)

    def update_high_price(self, value:float):
        if self.status:
            # self.high_price = max(self.high_price, value)
            self.high_price = value
            
    def update_entry_price(self, value:float):
        if not self.status:
            self.entry_price = value
            self.status = True
            self.high_price = value
            self.low_price = value

    def stop(self):
        if self.status:
            target_price = (self.low_price * (1 + self.target_roi))
            is_stop = target_price < self.high_price
            # print(f" << {target_price:,.2f} / {self.low_price:,.2f}")
            if is_stop:
                self.trade_history.append([self.entry_price, self.high_price])
                self.clear()
            return is_stop

    def clear(self):
        self.status = False
        self.entry_price = None
        self.high_price = None
        self.low_price = None

In [21]:
import Workspace.Utils.BaseUtils as base_utils

def sma_analysis(index:int, *periods):
    indices_1m = storage_indices.get_data("1m", index)
    indices_3m = storage_indices.get_data("3m", index)
    indices_5m = storage_indices.get_data("5m", index)

    data_1m = storage_closing.get_data('1m', indices_1m)
    data_3m = storage_closing.get_data('3m', indices_3m)
    data_5m = storage_closing.get_data('5m', indices_5m)

    sma_1m = cals_sma(data_1m[:, 4], *periods)
    sma_3m = cals_sma(data_3m[:, 4], *periods)
    sma_5m = cals_sma(data_5m[:, 4], *periods)
    
    closed_price = data_1m[:,4][-1]
    
    up_m1 = closed_price < sma_1m[7][-1] < sma_1m[25][-1] < sma_1m[99][-1]
    up_m3 = closed_price < sma_3m[7][-1] < sma_3m[25][-1] < sma_3m[99][-1]
    up_m5 = closed_price < sma_5m[7][-1] < sma_5m[25][-1] < sma_5m[99][-1]

    ema_m1_99 = np.all(np.diff(sma_1m[99])[-1] < 0)
    ema_m3_99 = np.all(np.diff(sma_3m[99])[-1] < 0)
    ema_m5_99 = np.all(np.diff(sma_5m[99])[-1] < 0)

    return all([up_m1, up_m3, up_m5, ema_m1_99, ema_m3_99, ema_m5_99]), data_1m

history = Order_L(0.025)

index = 0
periods = (7, 25, 99)

is_run = True
while is_run:
    # print(1)
    try:
        result, data = sma_analysis(index, *periods)
        
        low_price = data[-1][4]
        high_price = data[-1][2]
        close_price= data[-1][4]
        
        history.update_high_price(high_price)
        history.update_low_price(low_price)

        if result and not history.status:
            print(base_utils.convert_to_datetime(data[-1][0]))
            # base_utils.std_print(base_utils.convert_to_datetime(data[-1][0]))
            history.update_entry_price(close_price)
        history.stop()
    except:
        is_run = False
    index += 1

2025-01-08 12:40:00
2025-01-09 04:54:00
2025-01-10 04:48:00
2025-01-10 23:54:00
2025-01-11 10:42:00
2025-01-15 01:45:00
2025-01-16 12:36:00
2025-01-18 10:47:00
2025-01-19 18:15:00
2025-01-20 07:36:00
2025-01-21 00:18:00
2025-01-21 02:14:00
2025-01-21 04:31:00
2025-01-21 14:36:00
2025-01-22 10:37:00
2025-01-24 09:53:00
2025-01-25 05:45:00
2025-01-28 03:00:00
2025-01-28 21:31:00
2025-01-29 22:05:00


In [24]:
data = []
for a, b in history.trade_history:
    diff = a-b
    
    cals = diff / a
    data.append(cals)
    print(f"cals: {cals:,.2f}")

cals: 0.01
cals: 0.00
cals: -0.01
cals: -0.01
cals: 0.03
cals: -0.03
cals: -0.01
cals: -0.01
cals: -0.02
cals: 0.01
cals: 0.00
cals: 0.01
cals: 0.00
cals: -0.02
cals: 0.02
cals: -0.02
cals: 0.04
cals: -0.02
cals: -0.00
cals: -0.02


In [16]:
signal = False
if not signal:
    print(True)

True
